In [14]:
!pip install prophet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 31.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 14.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fbprophet 0.7.1 requires setuptools-git>=1.2, which is not installed.
fbprophet 0.7.1 requires cmdstanpy==0.9.5, but you have cmdstanpy 1.1.0 which is incompatible.


In [15]:
import pandas as pd
from pymongo import MongoClient
from prophet import Prophet

In [6]:
mongo_uri = "mongodb://hadoop-vm.internal.cloudapp.net:27017/ca2"

client = MongoClient(mongo_uri)
db = client["ca2"]
collection = db["vaccin_tweets_analysis"]

# Convert MongoDB collection to pandas DataFrame
df = pd.DataFrame(list(collection.find()))


In [10]:
df = df[["timestamp","sentiment_score"]]
df.rename(columns={"timestamp": "ds", "sentiment_score": "y"},inplace=True)

In [ ]:
# Prophet model

In [12]:
m = Prophet()
m.fit(df)

NameError: name 'Prophet' is not defined